# 8.2 Machine Learning: Automating and Consuming

Module 8 - AI in the Cloud

For book, references and training materials, please check this project website [http://activefitness.ai/ai-in-sports-with-python](http://activefitness.ai/ai-in-sports-with-python).

Book: [Applied Machine Learning for Health and Fitness](https://www.apress.com/us/book/9781484257715), Chapters 11-12

## Project: Registering, Deploying and Comsuming Models in the Cloud



In [1]:
import azureml.core
azureml.core.VERSION

'1.1.5'

In [ ]:
import azureml.core
from azureml.core import Workspace

workspace = Workspace.from_config()

In [15]:
# initial experiment configuration
experiment_name = 'activity-classification'
script_folder = 'activity-classification'
cluster_name = "pipeline-cluster"
model_file_name = 'activities.pkl'
labeled_dataset_name = 'Classifying activities-2020-03-15 00:54:26'
output_folder =  'outputs'
local_download_folder = './download/' 
env_name = 'pipeline-env'
experiment_name = 'pipeline-experiment'
model_name = 'activities'

In [3]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

auth = InteractiveLoginAuthentication(tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47')
workspace = Workspace.from_config(auth = auth)

In [9]:
from azureml.core.model import Model

model = Model.register(model_path = "./models",
                       model_name = "activities_classifier",
                       description = "Activity Classification",
                       tags = {'area':'classification'},
                       workspace = workspace)

Registering model activities_classifier


In [10]:
model = Model(workspace, 'activities_classifier')
print(model.name, model.version, model.tags)

activities_classifier 3 {'area': 'classification'}


In [12]:
Model.get_model_path('activities_classifier', _workspace=workspace)

'azureml-models\\activities_classifier\\3\\models\\activities.pkl'

## Creating a scoring script


In [16]:
%%writefile $script_folder/score.py

import torch
import torch.nn as nn
from torchvision import transforms
import json
import base64
from io import BytesIO
from PIL import Image
import os
import pickle

from azureml.core.model import Model

def transform(image_file):
    t = transforms.Compose([transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(), 
        transforms.Normalize(mean = [0.485, 0.456, 0.406], 
        std = [0.229, 0.224, 0.225])])
    image = Image.open(image_file)
    image = t(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image

def decode_base64_to_img(base64_string):
    base64_image = base64_string.encode('utf-8')
    decoded_img = base64.b64decode(base64_image)
    return BytesIO(decoded_img)

def init():
    global model, classes
    #model_path = Model.get_model_path('activities')
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'models', 'activities.pkl')
    model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.eval()
    #pkl_file = open(os.path.join(model_path,'class_names.pkl'), 'rb')
    #classes = pickle.load(pkl_file)
    #pkl_file.close() 
    classes = ['surfing','tennis']

def run(input_data):
    image = decode_base64_to_img(json.loads(input_data)['data'])
    image = transform(image)

    output = model(image)

    softmax = nn.Softmax(dim=1)
    pred_probs = softmax(model(image)).detach().numpy()[0]
    index = torch.argmax(output, 1)

    result = json.dumps({"label": classes[index], "probability": str(pred_probs[index])})
    return result

Overwriting activity-classification/score.py


Defining an environment
=======================



In [17]:
%%writefile $script_folder/activities.yml
name: Activities-PyTorch
dependencies:
  - python=3.6.2
  - pip:
    - azureml-defaults
    - azureml-core
    - azureml-contrib-dataset
    - azureml-dataprep[pandas,fuse]
    - inference-schema[numpy-support]
    - torch
    - torchvision
    - pillow

Overwriting activity-classification/activities.yml


In [18]:
from azureml.core.environment import Environment

env = Environment.from_conda_specification(name='Activities-PyTorch',file_path=script_folder+"/activities.yml")
env.register(workspace=workspace)

{
    "name": "Activities-PyTorch",
    "version": "2",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults",
                        "azureml-core",
                        "azureml-contrib-dataset",
                        "azureml-dataprep[pandas,fuse]",
                        "inference-schema[numpy-support]",
                        "torch",
                        "torchvision",
                        "pillow"
                    ]
                }
            ],
            "name": "azureml_29be90bc0029fbe93f78eab2d8a6383f"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage"

In [26]:
# List environments availablel in the cloud
envs = Environment.list(workspace=workspace)
for env in list(envs)[0:4]:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-TensorFlow-2.0-CPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.2.0
  - azureml-defaults==1.2.0
  - azureml-telemetry==1.2.0
  - azureml-train-restclients-hyperdrive==1.2.0
  - azureml-train-core==1.2.0
  - tensorflow==2.0
  - horovod==0.18.1
name: azureml_a685c8fa2729bbbf4932e75b8eb0df54

Name AzureML-Chainer-5.1.0-GPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.2.0
  - azureml-defaults==1.2.0
  - azureml-telemetry==1.2.0
  - azureml-train-restclients-hyperdrive==1.2.0
  - azureml-train-core==1.2.0
  - chainer==5.1.0
  - cupy-cuda90==5.1.0
  - mpi4py==3.0.0
name: azureml_43ae3494b9b7666919116b4a25139bcf

Name AzureML-VowpalWabbit-8.8.0
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.2.0
  - azureml-defaults==1.2.0
  - azureml-dataprep[fuse,pandas]
name: azureml_eed6129d1cdd3d18a4f0f2b746ad4d83



Deploying models
================



In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import LocalWebservice

def deploy_locally(model_name, port):
    model = Model(workspace, model_name)
    myenv = Environment.from_conda_specification(name="env", file_path=script_folder+"/activities.yml")
    inference_config = InferenceConfig(entry_script=script_folder+"/score.py", environment=myenv)
    deployment_config = LocalWebservice.deploy_configuration(port=port)
    return Model.deploy(workspace, model_name, [model], inference_config, deployment_config)

service = deploy_locally('activities', 8891)
service.wait_for_deployment(True)
print(service.port)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

service_name = 'activity-classification'
model = Model(workspace, 'activities')
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(workspace, service_name, [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

## Calling your model


In [ ]:
import json
import base64

image_path = 'download/workspaceblobstore/activities/surfing/resize-DSC04631.JPG'

with open(image_path, 'rb') as file:
    byte_content = file.read()
    
base64_bytes = base64.b64encode(byte_content)
base64_string = base64_bytes.decode('utf-8')
request = json.dumps({'data': base64_string })
prediction = service.run(input_data=request)
print(prediction)

In [ ]:
print("Model inference URI: ", service.scoring_uri)

## Creating a continuous model training pipeline

### Runtime environment 


In [28]:
import azureml.core
from azureml.core import Workspace, Dataset, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies

# Create environment and runtime configuration
environment = Environment(env_name)
environment.docker.enabled = True
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk',
                                                                        'azureml-contrib-dataset',
                                                                        'torch','torchvision',
                                                                        'azureml-dataprep[pandas,fuse]'])

In [31]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

print(compute_target.get_status().serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-03-24T16:59:21.234000+00:00', 'errors': None, 'creationTime': '2020-03-24T13:33:40.051086+00:00', 'modifiedTime': '2020-03-24T13:33:56.048041+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D3_V2'}


In [33]:
config = RunConfiguration()
config.target = compute_target
config.environment = environment

In [29]:
dataset = Dataset.get_by_name(workspace, labeled_dataset_name)
model_folder = PipelineData("model_folder", datastore=workspace.get_default_datastore())

### Creating training step 


In [34]:
estimator = Estimator(source_directory=script_folder, 
                      compute_target=compute_target,
                      environment_definition=config.environment,
                      entry_script='train.py')

train_step = EstimatorStep(name = "Train Model Step",
                           estimator=estimator, 
                           estimator_entry_script_arguments=['--output-folder', model_folder, '--model-file', model_file_name],
                           outputs=[model_folder],
                           compute_target=compute_target,
                           inputs=[dataset.as_named_input('activities')],
                           allow_reuse = True)

In [35]:
%%writefile $script_folder/register_model.py
import argparse
from azureml.core import Workspace, Model, Run

parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, dest='model_name', default="activities", help='Model name.')
parser.add_argument('--model_folder', type=str, dest='model_folder', default="outputs", help='Model folder.')
parser.add_argument('--model_file', type=str, dest='model_file', default="activities.pkl", help='Model file.')
args = parser.parse_args()
model_name = args.model_name
model_folder = args.model_folder
model_file = args.model_file

run = Run.get_context()

print("Model folder:",model_folder)
print("Model file:",model_file)

Model.register(workspace=run.experiment.workspace,
               model_name = model_name,
               model_path = model_folder+"/"+model_file)

run.complete()

Overwriting activity-classification/register_model.py


Defining deployment step
------------------------



In [36]:
register_step = PythonScriptStep(name = "Register Model Step",
                                source_directory = script_folder,
                                script_name = "register_model.py",
                                arguments = ['--model_name', model_name, '--model_folder', model_folder, '--model_file', model_file_name],
                                inputs=[model_folder],
                                compute_target = compute_target,
                                runconfig = config,
                                allow_reuse = True)

In [37]:
steps = [train_step, register_step]
print("Steps created")

pipeline = Pipeline(workspace=workspace, steps=steps)
print ("Pipeline created")

Steps created
Pipeline created


Running the pipeline
--------------------



In [ ]:
from azureml.core import Experiment

pipeline_run = Experiment(workspace, experiment_name).submit(pipeline)
pipeline_run.wait_for_completion()